In [1]:
from pyspark.sql import SparkSession

# Inicia una sesión Spark.
spark = SparkSession.builder.appName("ParteA2").getOrCreate()

# Lee el archivo CSV desde S3.
# Carga el archivo CSV sin encabezados.
df = spark.read.csv("s3://qqa-completo/05-2023_01.csv", header=False, inferSchema=True)

# Renombrar las columnas del DataFrame.
df_with_headers = df.toDF("producto", "presentacion", "marca", "categoria", "catalogo", "precio", "fecha_registro", "cadena_comercial", "giro", "nombre_comercial", "direccion", "estado", "municipio", "latitud", "longitud")

# Muestra las primeras filas con los nuevos encabezados.
df_with_headers.show()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
2,application_1713636313292_0003,pyspark,idle,Link,Link,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------------+--------------------+--------------------+--------------------+-----------------+-------+--------------+----------------+--------------------+--------------------+--------------------+--------------+--------------+---------+-----------+
|          producto|        presentacion|               marca|           categoria|         catalogo| precio|fecha_registro|cadena_comercial|                giro|    nombre_comercial|           direccion|        estado|     municipio|  latitud|   longitud|
+------------------+--------------------+--------------------+--------------------+-----------------+-------+--------------+----------------+--------------------+--------------------+--------------------+--------------+--------------+---------+-----------+
| ASISTENTES DE VOZ|ECHO SHOW 5. COLO...|               ALEXA|APARATOS ELECTRON...|ELECTRODOMESTICOS| 2499.0|    2023-05-02|          COPPEL|TIENDA DE ELECTRO...|COPPEL SUCURSAL E...|TECNOLOGICO 120, ...|AGUASCALIENTES|AGUASCALIE

In [2]:
# Guarda el CSV como parquet en S3, particionalo por catalogo, acomodar la salida de donde se almacenarán los datos particionados
df_with_headers.write.partitionBy("catalogo").mode("overwrite").parquet("s3://qqa-completo/output2/")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
# Carga del parquet en Spark
parquet_df = spark.read.parquet("s3://qqa-completo/output2/")

# Muestra las primeras filas.
parquet_df.show()

# Guarda la salida en el bucket de S3, apuntar la salida de donde quedarán los datos almacenados.
parquet_df.write.mode("overwrite").parquet("s3://qqa-completo/respuestasA2/parquet_df")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------------+--------------------+--------------------+--------------------+-------+--------------+----------------+--------------------+--------------------+--------------------+--------------+--------------+---------+-----------+-----------------+
|          producto|        presentacion|               marca|           categoria| precio|fecha_registro|cadena_comercial|                giro|    nombre_comercial|           direccion|        estado|     municipio|  latitud|   longitud|         catalogo|
+------------------+--------------------+--------------------+--------------------+-------+--------------+----------------+--------------------+--------------------+--------------------+--------------+--------------+---------+-----------+-----------------+
| ASISTENTES DE VOZ|ECHO SHOW 5. COLO...|               ALEXA|APARATOS ELECTRON...| 2499.0|    2023-05-02|          COPPEL|TIENDA DE ELECTRO...|COPPEL SUCURSAL E...|TECNOLOGICO 120, ...|AGUASCALIENTES|AGUASCALIENTES|21.879490|-10

In [4]:
# ¿Cuańtas marcas diferentes tiene tu categoría?

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [5]:
from pyspark.sql.functions import col

# Filtra por catalogo específica, por ejemplo 'Electrónica'
filtered_df = parquet_df.filter(col("catalogo") == "BASICOS")

# Selecciona la columna 'marca', encuentra valores únicos, y cuenta
unique_brands_count = filtered_df.select("marca").distinct().count()

print(f"Numero de marcas unicas en la catalogo: 'Basicos': {unique_brands_count}")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Numero de marcas unicas en la catalogo: 'Basicos': 523

In [6]:
# ¿Cuál es la marca con mayor precio? ¿En qué estado?

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [7]:
from pyspark.sql.functions import col, max

# Encuentra la marca con el precio más alto y el estado correspondiente
max_price_info = filtered_df.orderBy(col("precio").desc()).select("marca", "precio", "estado").first()

print(f"La marca con el precio más alto es {max_price_info['marca']} con un precio de {max_price_info['precio']} en el estado de {max_price_info['estado']}.")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

La marca con el precio m?s alto es DIVA CUP 2 con un precio de 690.0 en el estado de JALISCO.

In [8]:
# ¿Cuál es la marca con menor precio en CIUDAD DE MÉXICO?

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [9]:
# Filtra por estados que contengan la secuencia 'CIUDAD DE'
df_cdmx = filtered_df.filter(col("estado").contains("CIUDAD DE"))

# Encuentra la marca con el precio más bajo y el estado correspondiente
min_price_info = df_cdmx.orderBy(col("precio").asc()).select("marca", "precio", "municipio").first()

print(f"La marca con el precio más bajo en Ciudad de México es {min_price_info['marca']} con un precio de {min_price_info['precio']}.")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

La marca con el precio m?s bajo en Ciudad de M?xico es S/M con un precio de 1.6.

In [10]:
# ¿Cuál es la marca con mayores observaciones?

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [11]:
# Agrupa por la columna 'marca' y cuenta el número de observaciones para cada marca
brand_counts = filtered_df.groupBy("marca").count()

# Ordena los resultados por el conteo en orden descendente y toma la primera fila
most_observed_brand = brand_counts.orderBy(col("count").desc()).first()

# Verifica si hay resultados antes de acceder al resultado
if most_observed_brand:
    print(f"La marca con el mayor número de observaciones es {most_observed_brand['marca']}, con {most_observed_brand['count']} observaciones.")
else:
    print("No se encontraron registros en el dataset.")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

La marca con el mayor n?mero de observaciones es S/M, con 16231 observaciones.

In [12]:
# ¿Cuáles son el top 5 de marcas con mayor precio en cada estado? ¿Son diferentes?

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [13]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, row_number, collect_list
from pyspark.sql.window import Window

# Definir una ventana ordenada por precio descendente dentro de cada estado
windowSpec = Window.partitionBy("estado").orderBy(col("precio").desc())

# Aplicar la ventana con un número de fila para identificar los 5 principales
state_brand_prices = filtered_df.withColumn("row_number", row_number().over(windowSpec)) \
    .filter(col("row_number") <= 5) \
    .select("estado", "marca", "precio", "row_number")

state_brand_prices.show()

# Para comparar las 5 listas principales en diferentes estados, las recopilamos e imprimimos agrupadas por estado.
top_brands_by_state = state_brand_prices.groupBy("estado") \
    .agg(collect_list("marca").alias("top_5_brands")) \
    .orderBy("estado")
top_brands_by_state.show(50, False)


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------------------+--------------------+------+----------+
|             estado|               marca|precio|row_number|
+-------------------+--------------------+------+----------+
|     AGUASCALIENTES|                 S/M| 570.0|         1|
|     AGUASCALIENTES|                 S/M| 550.0|         2|
|     AGUASCALIENTES|HUGGIES. ULTRA CO...| 485.0|         3|
|     AGUASCALIENTES|JOSE CUERVO TRADI...| 470.0|         4|
|     AGUASCALIENTES|JOSE CUERVO TRADI...| 470.0|         5|
|    BAJA CALIFORNIA|                 S/M| 499.0|         1|
|    BAJA CALIFORNIA|JOSE CUERVO TRADI...| 469.0|         2|
|    BAJA CALIFORNIA|JOSE CUERVO TRADI...| 469.0|         3|
|    BAJA CALIFORNIA|HUGGIES. ULTRA CO...| 419.0|         4|
|    BAJA CALIFORNIA|HUGGIES. ULTRA CO...| 419.0|         5|
|BAJA CALIFORNIA SUR|                 S/M| 685.0|         1|
|BAJA CALIFORNIA SUR|                 S/M| 685.0|         2|
|BAJA CALIFORNIA SUR|          DIVA CUP 1| 584.0|         3|
|BAJA CALIFORNIA SUR|   

In [14]:
# ¿Cuáles son el top 5 de marcas con menor precio en CIUDAD DE MÉXICO?

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [15]:
# NOTA: use Contine por que no limpie los acentos
# Define una ventana ordenada por precio ascendente dentro de cada estado que contiene "CIUDAD DE".
windowSpec = Window.partitionBy("estado").orderBy(col("precio").asc())

# Aplica la especificación de ventana con un número de fila para identificar el top 5 de precios más bajos.
lowest_price_brands = filtered_df.withColumn("row_number", row_number().over(windowSpec)) \
    .filter(col("row_number") <= 5) \
    .select("estado", "marca", "precio", "row_number")

# Muestra las 5 marcas con los precios más bajos en los estados que contienen "CIUDAD DE".
lowest_price_brands.show()

# Para comparar las listas del top 5 entre diferentes "CIUDAD DE", las recopilamos y las mostramos agrupadas por estado.
top_brands_by_state_lowest = lowest_price_brands.groupBy("estado") \
    .agg(collect_list("marca").alias("top_5_lowest_brands")) \
    .orderBy("estado")
top_brands_by_state_lowest.show(50, False)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------------------+------+------+----------+
|             estado| marca|precio|row_number|
+-------------------+------+------+----------+
|     AGUASCALIENTES|   S/M|   1.6|         1|
|     AGUASCALIENTES|   S/M|   1.6|         2|
|     AGUASCALIENTES|   S/M|   1.8|         3|
|     AGUASCALIENTES|   S/M|   2.9|         4|
|     AGUASCALIENTES|   S/M|   2.9|         5|
|    BAJA CALIFORNIA|LIVEAN|   3.5|         1|
|    BAJA CALIFORNIA|CLIGHT|   3.6|         2|
|    BAJA CALIFORNIA|LIVEAN|   3.6|         3|
|    BAJA CALIFORNIA|CLIGHT|   3.6|         4|
|    BAJA CALIFORNIA|CLIGHT|   3.7|         5|
|BAJA CALIFORNIA SUR|   S/M|   1.8|         1|
|BAJA CALIFORNIA SUR|   S/M|   1.8|         2|
|BAJA CALIFORNIA SUR|   S/M|   1.8|         3|
|BAJA CALIFORNIA SUR|   S/M|   2.1|         4|
|BAJA CALIFORNIA SUR|   S/M|   2.1|         5|
|           CAMPECHE|   S/M|   1.3|         1|
|           CAMPECHE|   S/M|   1.3|         2|
|           CAMPECHE|   S/M|   1.3|         3|
|           C

In [16]:
# ¿Cuáles son el top 5 de marcas con mayores observaciones? ¿Se parecen a las de nivel por estado?

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [17]:
from pyspark.sql.functions import collect_list, array_contains

# Contar las ocurrencias de cada marca en todo el dataset
brand_occurrences = filtered_df.groupBy("marca").count()

# Ordenar las marcas por el número de observaciones en orden descendente y seleccionar el top 5
top_5_brands_observations = brand_occurrences.orderBy(col("count").desc()).limit(5)

# Mostrar el top 5 de marcas con mayores observaciones
top_5_brands_observations.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+-----+
|     marca|count|
+----------+-----+
|       S/M|16231|
|LA COSTEÑA| 7509|
|       FUD| 6177|
|     BIMBO| 5207|
|  SABRITAS| 4741|
+----------+-----+

In [18]:
from pyspark.sql.types import StringType  # Importa StringType aquí

# Buscar las marcas top 5 de observaciones en una lista
top_5_observation_list = [row['marca'] for row in top_5_brands_observations.collect()]

# Ahora creamos el DataFrame especificando el esquema
top_5_obs_df = spark.createDataFrame(top_5_observation_list, StringType()).withColumnRenamed("value", "marca")
top_5_obs_df.show()

# Realizar un join entre el DataFrame de top 5 por precio y el DataFrame de observaciones top 5
comparison_df = state_brand_prices.join(top_5_obs_df, "marca", "inner")

# Mostrar los resultados de la comparación
comparison_df.show()

# Comprobar cuántos estados tienen marcas que están en las top 5 de observaciones
distinct_states_with_top5_obs = comparison_df.select("estado").distinct().count()
print(f"Número de estados que tienen al menos una marca en común con las top 5 de observaciones: {distinct_states_with_top5_obs}")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+
|     marca|
+----------+
|       S/M|
|LA COSTEÑA|
|       FUD|
|     BIMBO|
|  SABRITAS|
+----------+

+-----+---------------+------+----------+
|marca|         estado|precio|row_number|
+-----+---------------+------+----------+
|  S/M|      ZACATECAS| 519.0|         3|
|  S/M|        YUCATÁN| 499.0|         3|
|  S/M|        YUCATÁN| 499.0|         2|
|  S/M|        YUCATÁN| 509.0|         1|
|  S/M|       VERACRUZ| 464.9|         5|
|  S/M|       VERACRUZ| 464.9|         4|
|  S/M|       VERACRUZ| 499.0|         3|
|  S/M|       VERACRUZ| 529.0|         1|
|  S/M|     TAMAULIPAS| 464.9|         5|
|  S/M|        TABASCO| 509.0|         2|
|  S/M|        TABASCO| 570.0|         1|
|  S/M|         SONORA| 489.0|         4|
|  S/M|         SONORA| 489.9|         3|
|  S/M|         SONORA| 489.9|         2|
|  S/M|         SONORA| 499.0|         1|
|  S/M|SAN LUIS POTOSÍ| 499.0|         5|
|  S/M|   QUINTANA ROO| 527.0|         4|
|  S/M|   QUINTANA ROO| 527.0|         3|


In [19]:
# ¿Ha dejado de existir alguna marca durante los años que tienes? ¿Cuál? ¿Cuándo desapareció?

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [20]:
# Extraer el año de la columna 'registro' que tiene formato DD-MM-YYYY
from pyspark.sql.functions import col, to_date, year

# Convertir la columna de registro a tipo fecha y extraer el año
filtered_df = filtered_df.withColumn("fecha", to_date(col("fecha_registro"), "dd-MM-yyyy"))
filtered_df = filtered_df.withColumn("fecha_y", year(col("fecha")))

# Mostrar las primeras filas para verificar
filtered_df.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+--------------------+----------------+--------------------+------+--------------+--------------------+--------------------+--------------------+--------------------+--------------+--------------+---------+-----------+--------+----------+-------+
|            producto|        presentacion|           marca|           categoria|precio|fecha_registro|    cadena_comercial|                giro|    nombre_comercial|           direccion|        estado|     municipio|  latitud|   longitud|catalogo|     fecha|fecha_y|
+--------------------+--------------------+----------------+--------------------+------+--------------+--------------------+--------------------+--------------------+--------------------+--------------+--------------+---------+-----------+--------+----------+-------+
|    TORTILLA DE MAIZ|        1 KG. GRANEL|             S/M|TORTILLAS Y DERIV...|  22.0|    2023-05-02|TORTILLERIAS TRAD...|        TORTILLERÍAS|TORTILLERIA 5 DE ...|EL GUARDA 128-A, ...|AGUASCALI

In [21]:
from pyspark.sql.window import Window
from pyspark.sql.functions import max as max_

# Ventana para particionar por marca
window_spec = Window.partitionBy("marca")

# Encontrar el último año de aparición para cada marca
last_appearance = filtered_df.withColumn("last_year", max_("fecha_y").over(window_spec))
last_appearance.show()


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+--------------------+---------------+---------------+------+--------------+--------------------+--------------------+--------------------+--------------------+---------------+--------------+---------+-----------+--------+----------+-------+---------+
|            producto|        presentacion|          marca|      categoria|precio|fecha_registro|    cadena_comercial|                giro|    nombre_comercial|           direccion|         estado|     municipio|  latitud|   longitud|catalogo|     fecha|fecha_y|last_year|
+--------------------+--------------------+---------------+---------------+------+--------------+--------------------+--------------------+--------------------+--------------------+---------------+--------------+---------+-----------+--------+----------+-------+---------+
|      LECHE EN POLVO|BOLSA 460 GR. ENTERA|ALPURA. CLÁSICA|LECHE PROCESADA|  73.0|    2023-05-02|            WAL-MART|SUPERMERCADO / TI...|WALMART SUCURSAL ...|INDEPENDENCIA 235...|

In [22]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, year, max as max_
from pyspark.sql.window import Window

# Seleccione columnas distintas de marca y último año para evitar filas duplicadas
distinct_last_appearance = last_appearance.select("marca", "last_year").distinct()


distinct_last_appearance.show()


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+---------+
|               marca|last_year|
+--------------------+---------+
|      ALPINO. PAVINO|     2023|
|     ALPURA. CLÁSICA|     2023|
|ALPURA. DESLACTOSADA|     2023|
|             BACHOCO|     2023|
|BAYGON. CASA Y JA...|     2023|
|    BORGES. ORIGINAL|     2023|
|           CAZADORES|     2023|
|         CHOCO CHOCO|     2023|
|              CLIGHT|     2023|
|    COCA COLA. LIGHT|     2023|
|   DANONE. VITALÍNEA|     2023|
|             DELICIA|     2023|
|              EFICAZ|     2023|
|         EL CUERNITO|     2023|
|           EL DORADO|     2023|
|             ENSUEÑO|     2023|
|            FABULOSO|     2023|
|           FLORIDA 7|     2023|
|GOICOECHEA. DIABE...|     2023|
|            GRANVITA|     2023|
+--------------------+---------+
only showing top 20 rows

# Genera una gráfica de serie de tiempo por estado para la marca con mayor precio -en todos los años-,
# donde el eje equis es el año y el eje ye es el precio máximo.

# NOTA: Este punto se hace aparte, se tiene que descargar la base de datos generda y se realizan las gráficas

In [24]:
from pyspark.sql.functions import col, max as max_, year

max_price_overall = filtered_df.select(max_("precio")).first()[0]
brand_max_price = filtered_df.filter(col("precio") == max_price_overall).select("marca").distinct().first()[0]

state_year_max_price = filtered_df.filter(col("marca") == brand_max_price) \
                         .groupBy("estado", "fecha_y") \
                         .agg(max_("precio").alias("max_price")) \
                         .orderBy("estado", "fecha_y")

# Collecting data directly
data_collected = state_year_max_price.collect()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [25]:
# Definir el bucket y la ruta
output_path = "s3://qqa-completo/respuestasA2/grafica de serie de tiempo.csv"

# Escribir el DataFrame en S3 como archivo tipo CSV
state_year_max_price.write.format("csv").option("header", "true").mode("overwrite").save(output_path)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…